In [1]:
import torch
from torch import nn

from torchvision import transforms as T
from torchvision.datasets import MNIST
from torchvision.utils import make_grid
from torch.utils.data import DataLoader

from tqdm.auto import tqdm
import matplotlib.pyplot as plt

In [2]:
def show_image_tensors(image_tensors, n_images=25, size=(1, 28, 28)):

  image_unflat = image_tensors.detach().cpu().view(-1, *size)
  image_grid = make_grid(image_unflat[:n_images], nrow=5)

  plt.imshow(image_grid.permute(1, 2, 0).squeeze())
  plt.show()

### Creating Generator

In [3]:
def get_generator_block(input_dim, output_dim):
  return nn.Sequential(
      nn.Linear(input_dim, output_dim),
      nn.BatchNorm1d(output_dim),
      nn.ReLU(inplace=True)
  )

In [4]:
class Generator(nn.Module):

  def __init__(self, z_dim=10, im_dim=784, hidden_dim=128):
    super(Generator, self).__init__()
    self.gen = nn.Sequential(
        get_generator_block(z_dim, hidden_dim),
        get_generator_block(hidden_dim, hidden_dim * 2),
        get_generator_block(hidden_dim * 2, hidden_dim * 4),
        get_generator_block(hidden_dim * 4, hidden_dim * 8),
        nn.Linear(hidden_dim * 8, im_dim),
        nn.Sigmoid()
    )

  def forward(self, noise):
    return self.gen(noise)

In [5]:
def get_noise(n_samples, z_dim, device='cpu'):
  return torch.randn(n_samples, z_dim, device=device)

### Creating Discriminator

In [13]:
def get_discriminator_block(input_dim, output_dim):
  return nn.Sequential(
      nn.Linear(input_dim, output_dim),
      nn.LeakyReLU(0.2, inplace=True)
  )

In [14]:
class Discriminator(nn.Module):

  def __init__(self, im_dim=784, hidden_dim=128):
    super(Discriminator, self).__init__()
    self.disc = nn.Sequential(
        get_discriminator_block(im_dim, hidden_dim * 4),
        get_discriminator_block(hidden_dim * 4, hidden_dim * 2),
        get_discriminator_block(hidden_dim * 2, hidden_dim),
        nn.Linear(hidden_dim , 1)
    )

  def forward(self, image):
    return self.disc(image)

### Training

In [9]:
loss_fn = nn.BCEWithLogitsLoss()
n_epochs = 200
z_dim = 64
display_step = 200
batch_size = 128
lr = 1e-5
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

In [11]:
dataloader = DataLoader(MNIST('.', download=True, transform=T.ToTensor()),
                        batch_size=batch_size,
                        shuffle=True)

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 9912422/9912422 [00:06<00:00, 1600464.93it/s]


Extracting ./MNIST/raw/train-images-idx3-ubyte.gz to ./MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 28881/28881 [00:00<00:00, 160198.28it/s]


Extracting ./MNIST/raw/train-labels-idx1-ubyte.gz to ./MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 1648877/1648877 [00:01<00:00, 1294776.87it/s]


Extracting ./MNIST/raw/t10k-images-idx3-ubyte.gz to ./MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 4542/4542 [00:00<00:00, 7574762.93it/s]


Extracting ./MNIST/raw/t10k-labels-idx1-ubyte.gz to ./MNIST/raw



In [15]:
gen = Generator(z_dim).to(device)
optimizer_G = torch.optim.Adam(params=gen.parameters(),
                               lr=lr)

disc = Discriminator().to(device)
optimizer_D = torch.optim.Adam(params=disc.parameters(),
                               lr=lr)